In [30]:
file = "/home/qtxu/Sentiment-SPN/data/Ele-COQE/train.txt"
file_write = "/home/qtxu/Sentiment-SPN/data/Ele-COQE/train_sentences_labels.txt"
# Open the file
with open(file, "r") as fr, open(file_write, "w") as fw:
  # Read the lines in the file
  lines = fr.readlines()

  for line in lines:
        try:
            sentence, label = line.strip().split("\t")
            fw.write(sentence+"\t"+label+"\n")
            # fw.write(sentence+"\n") # consider comparative sentence and non somparative
        except:
            assert "check again!"

In [7]:
import numpy as np
import spacy
import pickle
from transformers import BertTokenizerFast

from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split()
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)
tokenizer = BertTokenizerFast.from_pretrained("/home/qtxu/PLM/bert-base-uncased")

def dependency_adj_matrix(text):
    # https://spacy.io/docs/usage/processing-text
    tokens = nlp(text)
    tokenized = tokenizer(text.split(" "), is_split_into_words=True, add_special_tokens=False)
    word_ids = tokenized.word_ids()
    words = text.split()
    # matrix = np.zeros((len(words), len(words))).astype('float32')
    matrix1 = np.zeros((len(word_ids), len(word_ids))).astype('float32')
    assert len(words) == len(list(tokens))
    assert (len(tokens) - 1) == max(word_ids)

    for i, idx in enumerate(word_ids):
        matrix1[i][i] = 1
        for j, id in enumerate(word_ids):
            if tokens[id] in tokens[idx].children or word_ids[j] == word_ids[i]:
                matrix1[i][j] = 1
                matrix1[j][i] = 1
    return matrix1

def softmax(x):
    if len(x.shape) > 1:
        # matrix
        tmp = np.max(x, axis=1)
        x -= tmp.reshape((x.shape[0], 1))
        x = np.exp(x)
        tmp = np.sum(x, axis=1)
        x /= tmp.reshape((x.shape[0], 1))
    else:
        # vector
        tmp = np.max(x)
        x -= tmp
        x = np.exp(x)
        tmp = np.sum(x)
        x /= tmp
    return x

def process(filename):
    fin = open(filename, 'r', encoding='utf-8')
    lines = fin.readlines()
    fin.close()
    idx2graph = {}
    fout = open(filename+'.graph', 'wb')
    for i in lines:
        sentence = i.strip().split("\t")[0]
        try:
            adj_matrix = dependency_adj_matrix(sentence)
        except:
            print(filename)
            raise
        idx2graph[sentence] = adj_matrix
    pickle.dump(idx2graph, fout)
    fout.close()


# filename = "/home/qtxu/SPN/data/Camera-COQE/train_only_sentence.txt"
# process(filename)

max_lens = 12
sentence = "Both iXUS 40 and 65 are Made in Japan . "
adj_matrix = dependency_adj_matrix(sentence)
d_graph = np.pad(adj_matrix, ((1,max_lens-1-len(adj_matrix)), (1,max_lens-1-len(adj_matrix))), 'constant')
d_graph
type(d_graph)
# import pdb
# pdb.set_trace()
# print(adj_matrix)
# print(d_graph)


numpy.ndarray

In [9]:
import spacy
from spacy import displacy
 
nlp = spacy.load('zh_core_web_sm')
# 如果是对中文的句子进行解析，则使用'zh_core_web_sm'
# 注意无论是'en_core_web_sm'还是'zh_core_web_sm'的版本号都需要与spacy的版本保持一致
doc = nlp("钥匙是遥控的一体化钥匙，并能遥控后备箱。")
 
displacy.render(doc, style='dep', jupyter=True)

In [8]:
spacy_tokens = ['油耗', '比', '骑车', '要', '高', '。']
test_dict = {}
count = 0
for word in spacy_tokens:
    test_dict[count] = word
    count+=len(word)
test_dict

{0: '油耗', 2: '比', 3: '骑车', 5: '要', 6: '高', 7: '。'}

In [44]:
import spacy
nlp = spacy.load("zh_core_web_sm")

# 处理中文文本
text = "这是一段中文文本。"
from transformers import BertTokenizerFast
import numpy as np
tokenizer = BertTokenizerFast.from_pretrained("/home/qtxu/PLM/bert-base-chinese")
sen_tokens = tokenizer(text, add_special_tokens=False)
sen_tokens_input_ids = sen_tokens['input_ids']
sen_tokens_text_list = tokenizer.decode(sen_tokens['input_ids']).split(" ")
print(sen_tokens_text_list)
doc = nlp(text)

# 构造邻接矩阵
num_tokens = len(doc)
adjacency_matrix = np.zeros((len(sen_tokens_text_list), len(sen_tokens_text_list))).astype('float32')

# 遍历文本中的所有单独的词
for i, singel_token in enumerate(sen_tokens_text_list):
    # print("i is {}, singel_token is {}".format(i, singel_token))
    adjacency_matrix[i][i] =1
    for j, token in enumerate(doc):
        for child in token.children:
            adjacency_matrix[token.i][child.i] = 1

    
# for token in doc: # [这是 一 段 中文 文本]
#     print(token)
#     # 如果当前词有子节点，则在邻接矩阵中设置边
#     for child in token.children:
#         adjacency_matrix[token.i][child.i] = 1

# print(adjacency_matrix)


['这', '是', '一', '段', '中', '文', '文', '本', '。']


TypeError: argument of type 'spacy.tokens.token.Token' is not iterable

In [45]:
import spacy

nlp = spacy.load('zh_core_web_sm')

text = '这是一段中文文本'
print(len(text)) # 中文文本的句子长度
doc = nlp(text)

# 获取文本中的所有词
tokens = list(doc)
print(tokens) #[这是, 一, 段, 中文, 文本]

# 创建一个空的邻接矩阵
adjacency_matrix = [[0 for _ in range(len(text))] for _ in range(len(text))]

# 遍历所有词，填充邻接矩阵
for i, token in enumerate(tokens):
    print("i is {}, token is {}".format(i,token))
    adjacency_matrix[i][i] = 1 # 保留自身属性特征
    # 如果当前词是句子的根节点，则跳过
    stop()
    if token.dep_ == 'ROOT':
        continue
    # 如果当前词的父节点是句子中的其他词，则在邻接矩阵中填入 1
    if token.head.sent == token.sent:
        adjacency_matrix[i][token.head.i] = 1
        adjacency_matrix[token.head.i][i] = 1

# 打印邻接矩阵
print(adjacency_matrix)


8
[这是, 一, 段, 中文, 文本]
i is 0, token is 这是


NameError: name 'stop' is not defined

In [88]:
import re

def spacy_test_dict(text):
    nlp = spacy.load("zh_core_web_sm") # 加载spaCy模型
    doc = nlp(text)
    spacy_tokens = []
    for token in doc:
        spacy_tokens.append(token)
    # 依据spacy的分词解析结果，存放开始的index
    test_dict = {}
    count = 0
    for word in spacy_tokens:
        if word.like_num: # 匹配纯数字的情况，长度为1
            test_dict[count] = str(word)
            count += 1  
        elif re.findall(r"[a-zA-Z]", word.text): # 匹配纯字母的情况，长度为1
            test_dict[count] = str(word)
            count += 1
        else: # 正常的情况
            test_dict[count] = str(word)
            count+=len(word)
    return test_dict

def get_split(spacy_results, bert_results):
    not_complete_in_bert_results = {key:value for key, value in spacy_results.items() if 
    value not in bert_results and value not in ''.join(bert_results)} # 获取在bert中完全未出现的词语
    concatenated_values = []
    current_list = []

    previous_key = None
    for key in sorted(not_complete_in_bert_results.keys()):
        if previous_key is not None and key == previous_key + 1:
            current_list.append(not_complete_in_bert_results[key])
        else:
            if current_list:
                concatenated_values.append(''.join(current_list))
            current_list = [not_complete_in_bert_results[key]]
        previous_key = key

    if current_list:
        concatenated_values.append(''.join(current_list))
    return concatenated_values

sentence = "100-110KM/H以下,SAIL驾驶乐趣远大于富康,座椅人机工程设计我认为极为出色,十分难能可贵,SAIL就是为100KM(含)以下时速而设计的,德国人造车造神了,你不能不折服,我就是透过JETTA和SAIL认识德国车的。"
bert_path = "/home/qtxu/PLM/bert-base-chinese"
tokenizer = BertTokenizerFast.from_pretrained(bert_path)
tokenized = tokenizer.tokenize(sentence)
print(tokenized)
print(len(tokenized))

spacy_test_dict = spacy_test_dict(sentence) # 获取spacy解析的结果
print(spacy_test_dict)
list_result = get_split(spacy_test_dict, tokenized) # 获取bert tokenized 结果与spacy解析结果相比，spacy解析结果完全不在bert tokenized中的结果

print(list_result)
for i in range(len(list_result)): # 将合并之后的单词，用#替换
    sentence = sentence.replace(list_result[i], "#")
    
print(sentence) 


['100', '-', '[UNK]', '/', '[UNK]', '以', '下', ',', '[UNK]', '驾', '驶', '乐', '趣', '远', '大', '于', '富', '康', ',', '座', '椅', '人', '机', '工', '程', '设', '计', '我', '认', '为', '极', '为', '出', '色', ',', '十', '分', '难', '能', '可', '贵', ',', '[UNK]', '就', '是', '为', '[UNK]', '(', '含', ')', '以', '下', '时', '速', '而', '设', '计', '的', ',', '德', '国', '人', '造', '车', '造', '神', '了', ',', '你', '不', '能', '不', '折', '服', ',', '我', '就', '是', '透', '过', '[UNK]', '和', '[UNK]', '认', '识', '德', '国', '车', '的', '。']
90
{0: '100', 1: '-', 2: '110', 3: 'K', 4: 'M', 5: '/', 6: 'H', 7: '以下', 9: ',', 10: 'SAIL', 11: '驾驶', 13: '乐趣', 15: '远', 16: '大于', 18: '富康', 20: ',', 21: '座椅', 23: '人机', 25: '工程', 27: '设计', 29: '我', 30: '认为', 32: '极为', 34: '出色', 36: ',', 37: '十分', 39: '难能可贵', 43: ',', 44: 'SAIL', 45: '就是', 47: '为', 48: '100', 49: 'K', 50: 'M(', 51: '含)', 53: '以下', 55: '时速', 57: '而', 58: '设计', 60: '的', 61: ',', 62: '德国人', 65: '造车', 67: '造神', 69: '了', 70: ',', 71: '你', 72: '不能不', 75: '折服', 77: ',', 78: '我', 79: '就是', 81: '透过', 83: 

In [11]:
def get_split(spacy_results, bert_results):
    not_complete_in_bert_results = {key:value for key, value in spacy_results.items() if 
    value not in bert_results and value not in ''.join(bert_results)} # 获取在bert中完全未出现的词语
    concatenated_values = []
    current_list = []

    previous_key = None
    for key in sorted(not_complete_in_bert_results.keys()):
        if previous_key is not None and key == previous_key + 1:
            current_list.append(not_complete_in_bert_results[key])
        else:
            if current_list:
                concatenated_values.append(''.join(current_list))
            current_list = [not_complete_in_bert_results[key]]
        previous_key = key

    if current_list:
        concatenated_values.append(''.join(current_list))
    return concatenated_values

sent_tokened = ['感', '觉', '比', '[UNK]', '的', '抖', '动', '好', '得', '多', ',', '而', '且', '空', '调', '降', '温', '比', '[UNK]', '得', '还', '好', '.']
test_dict = {'感觉-0': '好得多-6', '比-1': '抖动-5', 'TJ-2': '抖动-5', '376-3': '抖动-5', '的-4': '376-3', '抖动-5': '好得多-6', '好得多-6': 'Root', ',-7': '好得多-6', '而且-8': '好得多-6', '空调-9': '降温-10', '降温-10': '而且-8', '比-11': '还好-15', 'LJ-12': '还好-15', '465-13': '还好-15', '得-14': '还好-15', '还好-15': '降温-10', '.-16': '还好-15'}
get_split(sent_tokened, test_dict)

'longer'

In [4]:

list1 = ['感觉', '比', 'TJ', '376', '的', '抖动', '好得多', ',', '而且', '空调', '降温', '比', 'LJ', '465', '得', '还好', '.']
import re
def count_len(stanford_words):
    sum_len = 0
    for i in range(len(list1)):
        if re.findall(r"\d+[a-zA-Z]+[\u4e00-\u9fff]+|\d|[a-zA-Z]|\d+[a-zA-Z]+[\u4e00-\u9fff]+|[\u4e00-\u9fff]+\d", list1[i]):
            sum_len += 1
        else:
            sum_len += len(list1[i])
    return sum_len

count_len(list1)

25

In [10]:
def dependency_adj_matrix_zh(sen, sent_tokened):
    stanford_words = list(zh_model.word_tokenize(sen))
    stan_dict = get_stan_dict(sen)
    test_dict = {}
    count = 0
    for word in stanford_words:
        if word.isdigit():
            test_dict[count] = word
            count += 1
        elif word == "DX":
            test_dict[count] = word
            count += 1  
        elif len(re.findall(r"\d+[a-zA-Z]+[\u4e00-\u9fff]+", word)) != 0: # 匹配数字字母汉字格式的str, eg 4s站
            test_dict[count] = word
            count += 1
        elif len(re.findall(r"\d+[\u4e00-\u9fff]+", word)) != 0: # 匹配数字汉字格式， eg:呈V
            test_dict[count] = word
            count +=1
        elif len(re.findall(r"[a-zA-Z]+\d+|\d+[.,-]+\d+", word)) != 0:  # 匹配字母数字格式，或数字标点符号数字格式 eg:5W， 25.4 or 11-13
            test_dict[count] = word
            count += 1
        elif len(re.findall(r"[\u4e00-\u9fff]+\d", word)) != 0: #匹配汉字数字格式，eg:
            test_dict[count] = word
            count += 1   
        elif re.findall(r"[a-zA-Z]", word):
            test_dict[count] = word
            count += 1         
        else: # 正常的情况
            test_dict[count] = word
            count+=len(word)
    # print("test_dict", test_dict)

    # number = 0
    adj_matrix = np.eye(len(sent_tokened)).astype('float32') # sent_tokened:表示的是经过berttokenfast分词之后的结果
    for key_i, value_i in stan_dict.items():
        try:
            key_word, key_id = key_i.rsplit("-", 1)
        except:
            print(key_i)
            # stop()
        try:
            value_word, value_id = value_i.rsplit('-', 1) # 不同于split("-"), 其可以处理11-13-13这样的特殊情况
        except:
            continue

        key_start = next(key for key, value in test_dict.items() if value == key_word)
        value_start = next(key for key, value in test_dict.items() if value == value_word)

        for i in range(key_start, key_start + len(key_word)): # 注意此处的id是test_dict中的id,而不是stan_dict中的id
            for j in range(value_start, value_start + len(value_word)):
                adj_matrix[i][j] = 1
                adj_matrix[j][i] = 1

    return adj_matrix

['TJ376', 'LJ465']

In [43]:
import re
diff_results = ['Cd', 'SHB022a', '12Xdvdr', '#', 'SEB026a', 'Cd', '32X']
sen_new = " dvd和Cd刻录方面，SHB022a支持12Xdvdr刻录和#刻录，与SEB026a类似，Cd刻录速度降32X。"
for diff in range(len(diff_results)):
    sen_new = sen_new.replace(diff_results[diff], "#") # 将解析的不同词语使用特殊符号#替换， 对应bert解析的unk

sen_new

' dvd和#刻录方面，#支持#刻录和#刻录，与#类似，#刻录速度降#。'

In [27]:
import os
import pickle
# file_name = os.path.join(args.data_path, f'{dep_mode_path}_sentences_labels.txt.graph') 
# file_name = "/home/qtxu/Sentiment-SPN/data/Car-COQE/train_sentences_labels.txt.graph"
file_name = "/home/qtxu/Sentiment-SPN/data/Car-COQE/train_test.txt.graph"
file_size = os.stat(file_name).st_size
if file_size == 0: # 增加一个判断条件，判断是否为空
    raise Exception("Error:File is empty.")
else:
    fgraph = open(file_name, "rb") # "rb"是文件打开模式。"r"表示读取模式，"b"表示二进制模式。
    idx2graph = pickle.load(fgraph)
    fgraph.close() # very import, sure added

text = "但坐进车内感觉又截然不同，似乎不是一个年代的车型，福美来显得很现代、很舒适，色调的搭配还营造出一种家庭的温馨，与菱帅的内饰水准明显不在一个档次上。　　"
# text = "与原来1.6宝来相比，还算可以接受。"
dependency_adj_matrix = idx2graph[text]


In [22]:
diff_results = ['207CC', '206CC']
# sen = "现在出产的幸福使者跟以前已经大不相同了，不能说装配和工艺上已经无懈可击，但绝对能够达到夏利以上的水平，不信你随便找出一辆型号为7100A的一汽佳星#幸福使者和一辆夏利打开发动机盖看一下，我可以肯定的说幸福使者的装配明显比夏利高一个档次。"
# sen = "207CC跑车的动力系统也较206CC更加强劲。"
sen = "但坐进车内感觉又截然不同，似乎不是一个年代的车型，福美来显得很现代、很舒适，色调的搭配还营造出一种家庭的温馨，与菱帅的内饰水准明显不在一个档次上。"
sen = " 款马6在配置方面拥有比新马6更实在一些。"
for diff in range(len(diff_results)):
    sen = sen.replace(diff_results[diff], "#") # 将解析的不同词语使用特殊符号#替换， 对应bert解析的unk
sen_new = sen
print(sen)
print(sen_new)


#跑车的动力系统也较#更加强劲。
#跑车的动力系统也较#更加强劲。
